In [1]:
# libraries required, please install pandas
import pandas as pd
from unidecode import unidecode
import re
import datetime
from datetime import date
import json
import numpy as np
import os
import time
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [8]:
base_path="/Users/sijiawu/Work/Thesis/Data/"
folder="031_proc_refs_full_set"


In [9]:
j_data=pd.read_pickle(base_path+"Combined/022_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.pkl")

post_1970s=pd.read_excel('../'+folder+'/refs_post_1970.xlsx')
cits_post_1970s=pd.read_excel('../'+folder+'/network_cit_post.xlsx')

post_1970s_v2=pd.read_excel('../'+folder+'/refs_post_1970_v2.xlsx')
cits_post_1970s_v2=pd.read_excel('../'+folder+'/network_cit_post_v2.xlsx')
post_1970s_v2['btch']=post_1970s_v2['btch']+'_v2'
post_1970s_v2['f_key']=post_1970s_v2['f_key']+'_v2'

pre_1970s=pd.read_excel('../'+folder+'/refs_pre_1970.xlsx')
cits_pre_1970s=pd.read_excel('../'+folder+'/network_cit_pre.xlsx')
pre_1970s=pre_1970s.rename(columns={'id': "ref_ord"})


In [7]:
j_data.columns

Index(['issue_url', 'author', 'title', 'journal', 'volume', 'number', 'pages',
       'year', 'ISSN', 'abstract', 'URL', 'publisher', 'content_type', 'type',
       'jid', 'author_split', 'urldate', 'reviewed-author', 'uploaded',
       'title_10', 'URL_og', 'number_og', 'title_og', 'author_og', 'pages_og',
       'j_fix', 'scopus_jid', 'scopus_id', 'scopus_authorgroup',
       'scopus_authors', 'scopus_affiliations', 'scopus_references',
       'scopus_author_full_names', 'scopus_title', 'scopus_year',
       'scopus_source_title', 'scopus_volume', 'scopus_issue', 'scopus_art_no',
       'scopus_page_start', 'scopus_page_end', 'scopus_page_count',
       'scopus_cited_by', 'scopus_doi', 'scopus_abstract', 'scopus_publisher',
       'scopus_document_type', 'scopus_publication_stage',
       'scopus_open_access', 'scopus_source', 'scopus_eid', 'scopus_title_og',
       'scopus_volume_og', 'scopus_issue_og', 'scopus_page_start_og',
       'scopus_page_end_og', 'scopus_year_og', 's_fix', 

In [13]:
j_data.dtypes

issue_url                    object
author                       object
title                        object
journal                      object
volume                       object
number                       object
pages                        object
year                          int64
ISSN                         object
abstract                     object
URL                          object
publisher                    object
content_type                 object
type                         object
jid                          object
author_split                 object
urldate                      object
reviewed-author              object
uploaded                    float64
title_10                     object
URL_og                       object
number_og                    object
title_og                     object
author_og                    object
pages_og                     object
j_fix                       float64
scopus_jid                   object
scopus_id                   

In [14]:
j_data['id_o']=j_data["URL"].str.split('/').str[-1]

In [15]:
pre_1970s['type'].value_counts()

type
3    15433
2    13810
4    12518
1     6705
8     3563
7      993
6      435
5       23
Name: count, dtype: int64

In [16]:
pre_sub=pre_1970s[['ref_ord', 'id_o','year_o', 'type',
       'author', 'title', 'journal', 'year', 'volume', 'issue', 'pages',
       'chapter_title', 'location', 'publisher', 'text_full', 'jstor',
       'journal_proc', 'year_proc', 'year_latest',
       'volume_proc', 'issue_proc', 'title_proc', 'match_url', 'outcome',
       'match_id', 'f_key']]


In [17]:
post_sub=post_1970s[['id_o', 'year_o','ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
        'f_key', 'year_proc',
       'year_latest', 'outcome', 'match_url', 'match_id']]


In [18]:
post_sub_v2=post_1970s_v2[['id_o', 'year_o','ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
        'f_key', 'year_proc',
       'year_latest', 'outcome', 'match_url', 'match_id']]

In [19]:
all_refs=pd.concat([pre_sub,post_sub,post_sub_v2 ],axis=0).reset_index(drop=True)

In [20]:
all_refs["match_id"]=all_refs["match_url"].str.split('/').str[-1]

In [21]:
print(all_refs.columns)
print(all_refs.shape)

Index(['ref_ord', 'id_o', 'year_o', 'type', 'author', 'title', 'journal',
       'year', 'volume', 'issue', 'pages', 'chapter_title', 'location',
       'publisher', 'text_full', 'jstor', 'journal_proc', 'year_proc',
       'year_latest', 'volume_proc', 'issue_proc', 'title_proc', 'match_url',
       'outcome', 'match_id', 'f_key', 'authors', 'month', 'full_reference',
       'status', 'alt_j'],
      dtype='object')
(534347, 31)


In [22]:
cit_network=all_refs[(all_refs['match_id'].isna()==False)&(all_refs['id_o']!=all_refs['match_id'])].reset_index(drop=True).drop_duplicates(subset= ['id_o', 'match_id'], keep='first')[['id_o','match_id', 'f_key']]

In [23]:
relevant=all_refs[all_refs['f_key'].isin(list(cit_network['f_key']))]

In [ ]:
j_data['id_o']=j_data['id_o'].astype(str)
cit_network['match_id']=cit_network['match_id'].astype(str)
cit_network['id_o']=cit_network['id_o'].astype(str)

version=str(datetime.datetime.now()).replace(' ','_')

j_data.to_excel('j_data_final_'+version+'.xlsx', index=False)
cit_network.drop_duplicates(subset=['id_o','match_id'],keep='first').to_excel('network_cit_1940_2020_'+version+'.xlsx', index=False)
all_refs.to_excel('refs_1940_2020_'+version+'.xlsx', index=False)
relevant.to_excel('refs_1940_2020_top5_'+version+'.xlsx', index=False)


In [25]:

j_data.to_pickle('j_data_final_'+version+'.pkl')
cit_network.drop_duplicates(subset=['id_o','match_id'],keep='first').to_pickle('network_cit_1940_2020_'+version+'.pkl')
all_refs.to_pickle('refs_1940_2020_'+version+'.pkl')
relevant.to_pickle('refs_1940_2020_top5_'+version+'.pkl')